In [1]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts


import itertools
from tqdm import tqdm



In [2]:
def training_intervals():
    run_ = scipy.io.loadmat('runintervals.mat')['runintervals']
    runintervals = nts.IntervalSet(run_[:,0],run_[:,1],time_units='s')
    return runintervals

In [277]:
def mean_firing(neurons,intervals):
    binSize = 1
    t,fr = bk.compute.binSpikes(neurons,binSize)
    t = nts.Ts(t,time_units='s')
    fr = np.mean(fr[:,intervals.in_interval(t)>= 0]/binSize,0)
    t = t[intervals.in_interval(t)>=0]
    fr = nts.Tsd(t.index.values, d = fr)
    fr = bk.compute.nts_smooth(fr,1000,1)
    return fr

In [278]:
bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()
pos = bk.load.pos()
train = training_intervals()
pos = pos.restrict(train)
speed = bk.compute.old_speed(pos,1)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [285]:
%matplotlib inline
stru = ['Hpc','BLA']
types = ['Pyr','Int']

for s,t in itertools.product(stru,types):
    print(s,t)
    neur = neurons[(metadata.Region == s) & (metadata.Type == t)]
    meta = metadata[(metadata.Region == s) & (metadata.Type == t)]
    for n,shank,id in zip(neur,meta['Shank'],meta['Id']):
        fr = mean_firing([n],train)
        stats = (scipy.stats.linregress(np.log(speed.realign(fr).values),np.log(fr.values)))
        if stats.pvalue <0.001:
            plt.figure()
            plt.scatter(np.log(speed.realign(fr).values),np.log(fr.values).dropnan,1)
            x = np.linspace(0,20,2500)
            y = stats.intercept + x*stats.slope
            plt.plot(x,y,'red')
            plt.title(s+t+' '+str(shank)+' '+str(id))
# plt.legend(list(itertools.product(stru,types)))

# plt.subplot(2,1,2,sharex = sub)
# plt.plot(speed)

Hpc Pyr


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


Hpc Int
BLA Pyr
BLA Int


KeyboardInterrupt: 

In [274]:
bk.load.current_session('Z:\\Rat08\\Rat08-20130710')
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

for i in np.unique(metadata['Shank']):
    print(i,len(neurons[metadata['Shank']==i]))

Rat : 8 on day : 3
Working with session Rat08-20130710 @ Z:\Rat08\Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy
1 1
2 22
3 8
5 13
6 7
7 23
8 13
9 2
10 6
11 13
12 13
13 1
14 7
15 9
16 5
17 7
18 8
19 11
20 21


In [287]:
a = np.array([np.nan])

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'